# Customviz - A concept notebook for creating custom configurations
This notebook provides a whiteboard workspace for developing/hacking the infrastructure required for aiding the development of custom configurations. This builds off of a preliminary investigation by Brett earlier on.

## A "ConfigHelper Helper"
Currently, our configurations are defined in YAML files that are "almost" hardcoded into the code. But all this is doing in the backend is reading the file, constructing a dictionary from the contents, and passing it down the line. So, if we had a helper to help us make our own dictionary, we should be able to pass that in instead and avoid the need to introduce a YAML file at all!

In [ ]:
# For reference, this is how the layout is currently retrieved for our configs:
from jdaviz.core.config import read_configuration
read_configuration('specviz')

In [ ]:
# A new prototype config constructor
# Currently missing: How to specify the specific layout of the viewers themselves. Currently only adds them to the same row
class jdaviz_config():
    default_viewer_names = {
        'spectrum1d': {'name': 'Spectrum', 'reference': 'spectrum-viewer'},
        'spectrum2d': {'name': '2D Spectrum viewer', 'reference': 'spectrum-2d-viewer'},
        'image' : {'name': 'Image-viewer', 'reference': 'image-viewer'},
        'table': {'name': 'Table viewer', 'reference': 'table-viewer'}
        #'time': {'name': 'Time-viewer', 'reference': 'time-viewer'}
    }

    default_plot_classes = {
        'spectrum1d': 'specviz-profile-viewer',
        'spectrum2d': 'mosviz-profile-2d-viewer',
        'image': 'mosviz-image-viewer',
        'table': 'mosviz-table-viewer'
        #'time': 'lcviz-time-viewer'
    }
    
    def __init__(self, config, plugins=None, tools=None, viewers=None):

        self.settings = {'configuration': config,
                         'visible': {'menu_bar': False,
                                     'toolbar': True,
                                     'tray': True,
                                     'tab_headers': False
                                    },
                         'dense_toolbar': False,
                         'context': {'notebook': {'max_height': '600px'}}
                        }
        
        self.viewers = []
        for viewer in viewers:
            self.add_viewer(viewer)         
        
        self.toolbar = []
        for tool in tools:
            self.toolbar.append(tool)

        self.plugins = []
        for plugin in plugins:
            self.plugins.append(plugin)
        
    
    def get_config(self):
        viewer_area = [
                       {'container': 'col',
                        'children' : [
                                      {'container': 'row',
                                       'viewers': self.viewers
                                      }
                                     ]
                       }
                      ]
        return {'settings'   : self.settings,
                'toolbar'    : self.toolbar,
                'tray'       : self.plugins,
                'viewer_area': viewer_area
               }
    
    
    def add_viewer(self, type, name=None, viewer_register=None, reference=None):
        '''
        type: str
            Specifies the type of viewer. Must be one of [spectrum1d, spectrum2d, image, table]
        '''
        if not name:
            name = self.default_viewer_names[type]['name']
        if not viewer_register:
            viewer_register = self.default_plot_classes[type]
        if not reference:
            reference = self.default_viewer_names[type]['reference']
        
        self.viewers.append({'name': name,
                             'plot': viewer_register,
                             'reference': reference
                            })
        
    
    def add_tool(self, tool):
        self.toolbar.append(tool)
    
    
    def add_plugin(self, plugin):
        self.config['tray'].append(plugin)

        
    def register_parser(self, parser):
        raise NotImplementedError
    
    
    def register_viewer(self, viewer, viewer_name):
        raise NotImplementedError
        

In [ ]:
# Here's how you would expect to call it manually
viewers=['image', 'spectrum1d']
tools=['g-subset-tools']
plugins=['g-metadata-viewer', 'g-plot-options']

x = jdaviz_config(config='customviz', viewers=viewers, tools=tools, plugins=plugins)
x.get_config()

In [ ]:
from jdaviz.core.helpers import ConfigHelper

class Customviz(ConfigHelper):
    
    _viewers = ['spectrum1d', 'image', 'table']
    
    _plugins = ['g-metadata-viewer', 'g-plot-options', 'HelloWorldPlugin']
    
    _tools = ['g-data-tools', 'g-subset-tools']
    
    _default_configuration = jdaviz_config(config='customviz', viewers=_viewers, tools=_tools, plugins=_plugins).get_config()
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._default_spectrum_viewer_reference_name = jdaviz_config.default_viewer_names['spectrum1d']['reference']

In [ ]:
customviz = Customviz()
customviz.show()